## Step 1: Fetch the real world data

In [2]:

import json
import requests
import codecs
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs

# fetching data from file
data = None
file = '../data/location1.json'
with open(file, "r") as read_file:
    data = json.load(codecs.open(file, 'r', 'utf-8-sig'))

coord = []

# preprocessing the data
for line in data:
    coord.append(line['geometry']['coordinates']) 
    
new = []
for line in coord:
    for points in line:
        new.append(points)
        
# Isolating the dependencies
new_coord = np.unique(np.array(coord),axis=0)
new_coord.shape
dat = pd.DataFrame(new_coord, columns=['x','y'])
dat.to_csv("real_housing_data_x_y.csv")

#### Defining our constants and variables

In [ ]:
# seed for consistent results
RANDOM_SEED = 32
np.random.seed(RANDOM_SEED)

NUM_HOUSES = len(dat)

MEAN_DEMAND_WATER_PER_HOUSE = 100 # LITERS
VAR_DEMAND_WATER_PER_HOUSE = 50 # LITERS

# This can be evaluated using the vision api
MEAN_AREA_PER_HOUSE = 100 # METERS SQUARE
VAR_AREA_PER_HOUSE = 50 # METERS SQUARE PER HOUSE

RAINFALL_ESTIMATE = 100 # mm
FC_TANK_PLACEMENT = 10000 # Rs.
FIXED_COST_PIPE_PER_METER = 120 # Rs.
COST_PER_LITERS_TANK = 500 # R

In [6]:
house_data_x = dat['x'].values
house_data_y = dat['y'].values

In [4]:
dat.columns

Index(['x', 'y'], dtype='object')